<a href="https://colab.research.google.com/github/mahdi-alalawi/AndroidTutorialForBeginners/blob/master/%D8%AA%D8%B9%D8%AF%D9%8A%D9%84%D8%A7%D8%AA_%D8%B9_%D8%A7%D9%84%D8%B1%D8%B3%D9%88%D9%85%D8%A7%D8%AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt

def plot_bollinger_bands(data, window=20, num_std_dev=2):
    """رسم بولنجر باندز على مخطط بياني."""
    # حساب المتوسط المتحرك والانحراف المعياري
    data['Middle_Band'] = data['close'].rolling(window=window).mean()
    data['Std_Dev'] = data['close'].rolling(window=window).std()

    # حساب الحد العلوي والحد السفلي
    data['Upper_Band'] = data['Middle_Band'] + (data['Std_Dev'] * num_std_dev)
    data['Lower_Band'] = data['Middle_Band'] - (data['Std_Dev'] * num_std_dev)

    # طباعة القيم
    print("Middle Band:", data['Middle_Band'].iloc[-1])
    print("Upper Band:", data['Upper_Band'].iloc[-1])
    print("Lower Band:", data['Lower_Band'].iloc[-1])

    # رسم المخطط
    plt.figure(figsize=(12, 6))
    plt.plot(data['close'], label='Close Price', color='blue')
    plt.plot(data['Middle_Band'], label='Middle Band', color='orange')
    plt.plot(data['Upper_Band'], label='Upper Band', color='green')
    plt.plot(data['Lower_Band'], label='Lower Band', color='red')
    plt.fill_between(data.index, data['Upper_Band'], data['Lower_Band'], color='lightgray', alpha=0.5)
    plt.title('Bollinger Bands')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

# مثال على كيفية استخدام الدالة
# تأكد من أن لديك بيانات في DataFrame باسم `data` تحتوي على عمود 'close'
# plot_bollinger_bands(data)


In [ ]:
import matplotlib.pyplot as plt

def plot_premium_discount(data):
    """رسم Premium وDiscount على مخطط بياني."""
    # تأكد من أن البيانات تحتوي على الأعمدة المطلوبة
    data['Premium'] = data['close'] - data['low'].rolling(window=20).min()
    data['Discount'] = data['high'].rolling(window=20).max() - data['close']

    # طباعة القيم الأخيرة
    print("Last Premium Value:", data['Premium'].iloc[-1])
    print("Last Discount Value:", data['Discount'].iloc[-1])

    # رسم المخطط
    plt.figure(figsize=(12, 6))
    plt.plot(data.index, data['Premium'], label='Premium', color='blue')
    plt.plot(data.index, data['Discount'], label='Discount', color='red')
    plt.fill_between(data.index, data['Premium'], color='lightblue', alpha=0.5)
    plt.fill_between(data.index, data['Discount'], color='lightcoral', alpha=0.5)
    plt.title('Premium and Discount Bands')
    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.legend()
    plt.show()

# مثال على كيفية استخدام الدالة
# تأكد من أن لديك بيانات في DataFrame باسم `data` تحتوي على الأعمدة 'close', 'low', 'high'
# plot_premium_discount(data)


In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import time
import gc
import matplotlib.pyplot as plt  # استيراد مكتبة الرسم البياني

# ... (بقية الكود كما هو)

def plot_data(data):
    """رسم بيانات الأسعار مع Bollinger Bands و RSI."""
    plt.figure(figsize=(14, 10))

    # رسم أسعار الذهب
    plt.subplot(3, 1, 1)
    plt.plot(data['close'], label='سعر الذهب', color='gold')
    plt.plot(data['Upper_Band'], label='الحد العلوي', linestyle='--', color='red')
    plt.plot(data['Lower_Band'], label='الحد السفلي', linestyle='--', color='green')
    plt.title('أسعار الذهب مع Bollinger Bands')
    plt.legend()

    # رسم RSI
    plt.subplot(3, 1, 2)
    plt.plot(data['RSI'], label='RSI', color='blue')
    plt.axhline(70, linestyle='--', alpha=0.5, color='red')
    plt.axhline(30, linestyle='--', alpha=0.5, color='green')
    plt.title('مؤشر القوة النسبية (RSI)')
    plt.legend()

    # رسم حجم التداول
    plt.subplot(3, 1, 3)
    plt.bar(data.index, data['volume'], label='حجم التداول', color='gray')
    plt.title('حجم التداول')
    plt.legend()

    plt.tight_layout()
    plt.show()

def build_strategy(data):
    """بناء الاستراتيجية الكاملة مع تحليل جميع المؤشرات."""
    try:
        # إعداد البيانات
        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        # حساب جميع المؤشرات
        data = calculate_rsi(data)  # حساب مؤشر القوة النسبية
        data = calculate_moving_average(data)  # حساب المتوسط المتحرك
        data = calculate_momentum(data)  # حساب الزخم
        premium, discount = calculate_premium_discount(data)  # حساب العلاوة والخصم
        data['Premium'] = premium
        data['Discount'] = discount

        data = calculate_bollinger_bands(data)
        data = calculate_macd(data)

        # حساب مناطق العرض والطلب
        supply_zones, demand_zones = calculate_supply_demand(data)
        support, resistance = calculate_support_resistance(data)
        data['Support'] = support
        data['Resistance'] = resistance

        # حساب مستويات فيبوناتشي
        fib_levels = fibonacci_retracement(data['high'].max(), data['low'].min())
        for key, value in fib_levels.items():
            data[key] = value

        # جلب الأخبار وتحليل المشاعر
        news_titles = fetch_news()
        sentiments = analyze_sentiment(news_titles)
        average_sentiment = np.mean(sentiments) if sentiments else 0
        print(f"Average Sentiment: {average_sentiment}")

        # رسم البيانات
        plot_data(data)  # استدعاء دالة الرسم

        # ... (بقية الكود كما هو)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")

# ... (بقية الكود كما هو)
